# SQL query examples
### This notebook contains example queries to the EFSA database we created. This is to provide an idea how the database structure works in practice.
Author: Wouter Hoenderdaal\
Edited by: Osman Mutlu

### Imagine a scenario where you are researching mycotoxins.
### Let's first look at what parameters does EFSA have in the database.

In [ ]:
-- We use this temporary table for later queries 
CREATE TABLE efsa.tmp_mycotoxines
AS
SELECT *
FROM ontologies_efsa.vw_param
WHERE totalname LIKE '%mycoto%'
;

-- nr of mycotoxine measurements
SELECT sm.filetype, sm.param, COUNT(*) AS n
FROM efsa.vw_sample_measurement_core_legallimit sm
JOIN efsa.tmp_mycotoxines t
	ON sm.param = t.termextendedname
GROUP BY sm.filetype, sm.param
ORDER BY COUNT(*) DESC;

### We may want to see this per sampling country...

In [ ]:
-- nr of mycotoxine measurements per sampcountry
SELECT sampcountry, COUNT(*) AS n
FROM efsa.vw_sample_measurement_core_legallimit sm
JOIN efsa.tmp_mycotoxines t
	ON sm.param = t.termextendedname
WHERE sm.filetype = 'chemical'
GROUP BY sampcountry
ORDER BY COUNT(*) DESC;

### Lastly, we may want to exclude samples that were suspicious for a more unbiased sampling for our research.

In [ ]:
-- nr of non-suspect mycotoxine measurements per sampcountry
SELECT sampcountry, COUNT(*) AS n
FROM efsa.vw_sample_measurement_core_legallimit sm
JOIN efsa.tmp_mycotoxines t
	ON sm.param = t.termextendedname
WHERE sm.filetype = 'chemical'
AND sm.sampstrategy = 'suspect sampling'
GROUP BY sampcountry
ORDER BY COUNT(*) DESC;

### Let's look at the comparison of chemical contaminant measurements for Germany and Netherlands.

In [ ]:
-- compare chemical contaminants between germany and NL
SELECT sampcountry, param, COUNT(*) AS n
FROM efsa.vw_sample_measurement_core_legallimit sm
WHERE sm.filetype = 'chemical'
AND sampcountry IN ('netherlands', 'germany')
GROUP BY sampcountry, param
ORDER BY param, COUNT(*) DESC;

### Again excluding suspicious samples...

In [ ]:
-- compare chemical contaminants between germany and NL excl suspect sampling
SELECT sampcountry, param, COUNT(*) AS n
FROM efsa.vw_sample_measurement_core_legallimit sm
WHERE sm.filetype = 'chemical'
AND sampcountry IN ('netherlands', 'germany')
AND sm.sampstrategy <> 'suspect sampling'
GROUP BY sampcountry, param
ORDER BY param, COUNT(*) DESC;

### Let's select only mycotoxins, since we are interested in them.

In [ ]:
-- compare mycotoxine contaminants between germany and NL excl suspect sampling
SELECT sampcountry,  param, COUNT(*) AS n
FROM efsa.vw_sample_measurement_core_legallimit sm
JOIN efsa.tmp_mycotoxines t
	ON sm.param = t.termextendedname
WHERE sm.filetype = 'chemical'
AND sampcountry IN ('netherlands', 'germany')
AND sm.sampstrategy <> 'suspect sampling'
GROUP BY sampcountry, param
ORDER BY COUNT(*) DESC;

### We only looked at the number of measurements so far. What about the measurements that caught above legal limit mycotoxin contaminants?
### Let's prepare a temporary table to use for this purpose.

In [ ]:
CREATE TEMPORARY TABLE tmp_mycos AS
SELECT 	*,
		CASE 
		WHEN evalcode IN ('greater than maximum permissible quantities', 'non compliant', 'unsatisfactory', 'detected') THEN 1
		ELSE 0
		END exceeding
		
FROM efsa.vw_sample_measurement_core_legallimit
WHERE filetype = 'chemical' 
AND param IN ('aflatoxin b1',
'ochratoxin a',
'deoxynivalenol',
'zearalenone',
'fumonisin b1',
'fumonisin b2',
'sum of fumonisin b1 + b2',
't-2 toxin',
'ht-2 toxin',
'sum t-2 and ht-2',
'zearalenol alpha',
'zearalanol',
'zearalanone')
;
-- we are interested in these particular mycotoxins

### Let's see the distribution of measurement results per country.

In [ ]:
-- evalcode
SELECT sampcountry, evalcode, COUNT(*) AS n
FROM tmp_mycos
GROUP BY sampcountry, evalcode
ORDER BY evalcode, COUNT(*) DESC
;

### How many are exceeding limits per country?

In [ ]:
-- perc_exceeding
SELECT sampcountry, n, n_exceeding, n_exceeding::numeric / n::numeric AS perc_exceeding
FROM (
	SELECT sampcountry, COUNT(*) AS n, SUM(exceeding) AS n_exceeding
	FROM tmp_mycos
	GROUP BY sampcountry
) AS t1
ORDER BY n_exceeding::numeric / n::numeric DESC
;

### What about the Netherlands particularly?

In [ ]:
SELECT sampy, n, n_exceeding, n_exceeding::numeric / n::numeric AS perc_exceeding
FROM (
	SELECT sampy, COUNT(*) AS n, SUM(exceeding) AS n_exceeding
	FROM tmp_mycos
	WHERE sampcountry = 'netherlands'
	GROUP BY sampy
) AS t1
ORDER BY sampy ASC
;